In [1]:
%autosave 60 

Autosaving every 60 seconds


In [2]:
#default_exp segmentation.losses_multilabel

In [3]:
import fastai; print(fastai.__version__)

1.0.58.dev0


In [4]:
#export
from fastai.vision import *
from local.segmentation.lovasz_loss import *

In [5]:
#export
_all_ = ["lovasz_softmax"]

In [6]:
# export
cross_entropy = CrossEntropyFlat(axis=1)

### test

In [7]:
from local.test import *

### `cross_entropy`

In [8]:
_input = torch.ones((1,2,3,3))

_target = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

_input.size(), _target.size()

(torch.Size([1, 2, 3, 3]), torch.Size([1, 1, 3, 3]))

In [9]:
test_close(cross_entropy(_input, _target), tensor(0.6931), eps=1e-4)

### `lovasz_softmax` -> (best:0, random:0.5, worst:1)

In [10]:
_input = torch.ones((1,2,3,3))

_input2 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*1e3

_input3 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*-1e3

_target = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

_input.size(), _target.size()

(torch.Size([1, 2, 3, 3]), torch.Size([1, 1, 3, 3]))

In [11]:
# random
(lovasz_softmax(_input, _target))

tensor(0.5000)

In [12]:
# best
(lovasz_softmax(_input2, _target))

tensor(0.)

In [13]:
# worst
(lovasz_softmax(_input3, _target))

tensor(1.)

### `bce_sigmoid`

Loss for **models that don't predict background** and use sigmoid + threshold instead of softmax/argmax. Useful for imbalanced class distributions, learning background is hard: see [Focal Loss](https://arxiv.org/pdf/1708.02002.pdf).

In [101]:
_input = torch.randn(2,4,3,3)
# 0: void
_target = tensor([
    [[
    [1,0,0],
    [0,1,2],
    [0,3,4]
]],
      [[
    [1,0,0],
    [0,1,2],
    [0,3,4]
]]

])

_input.size(), _target.size()

(torch.Size([2, 4, 3, 3]), torch.Size([2, 1, 3, 3]))

In [123]:
#export
def bce_sigmoid(input, target, **bce_kwargs):
    "Background is not predict, e.g. codes.txt and background value in label image is 0"
    nclasses = input.size(1)
    trange = tensor(np.arange(1, nclasses+1))[None,...,None,None].to(target.device)
    new_target = target == trange
    return F.binary_cross_entropy_with_logits(input, new_target.float(), **bce_kwargs)

In [124]:
bce_sigmoid(_input, _target)

tensor(0.8397)

### export

In [125]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.


### fin